In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import time
import sklearn
import scipy

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [3]:
heart = pd.read_csv('FinalHeart.csv')
heart.drop(['Unnamed: 0'],axis=1,inplace=True)
heart.head(3)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0.0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2.0
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1.0


In [4]:
X = heart.drop(['num'],axis=1).copy()
Y = heart.num.copy()

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size= 0.33, stratify=Y)

In [6]:
rfc = RandomForestClassifier()

In [7]:
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [8]:
rfc.set_params( n_estimators = 500)
%time rfc.fit(X_train, Y_train)
r2_train = np.round(rfc.score(X_train, Y_train),3)
r2_test  = np.round(rfc.score(X_test, Y_test),3)
print('Train score is :' + str(r2_train))
print('Test score is :' + str(r2_test))

CPU times: total: 1.41 s
Wall time: 1.72 s
Train score is :1.0
Test score is :0.622


In [9]:
#?rfc

In [11]:
criterion      = ['gini','entropy']
max_depth      = np.arange(1,20, 5)

#max_features   = list(np.arange(1,14,2))
max_features = list(['auto','sqrt'])

max_leaf_nodes = np.arange(1,20,5)
min_samples_leaf  = np.arange(1,20,5)
min_samples_split = np.arange(2,20,5)
#class_weight  = ['balanced','balanced_subsample','None']

n_estimators  = 500
n_jobs = -1
random_state = 144

In [12]:
rfc_grid = RandomForestClassifier()
rfc_grid.set_params(random_state = 144, n_estimators  = 500)

param_grid = {'criterion':criterion, 'max_depth':max_depth, 'max_features':max_features, 'max_leaf_nodes':max_leaf_nodes,
             'min_samples_leaf':min_samples_leaf, 'min_samples_split':min_samples_split}

rfc_grid = GridSearchCV(rfc_grid, param_grid, cv = 3, return_train_score=True)

%time rfc_grid.fit(X_train, Y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
768 fits failed out of a total of 3072.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
768 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "C:\ProgramData\Anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\ProgramData\Anaconda3\lib\site-package

CPU times: total: 40min 49s
Wall time: 43min 2s


GridSearchCV(cv=3,
             estimator=RandomForestClassifier(n_estimators=500,
                                              random_state=144),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([ 1,  6, 11, 16]),
                         'max_features': ['auto', 'sqrt'],
                         'max_leaf_nodes': array([ 1,  6, 11, 16]),
                         'min_samples_leaf': array([ 1,  6, 11, 16]),
                         'min_samples_split': array([ 2,  7, 12, 17])},
             return_train_score=True)

In [14]:
rfc_df = pd.DataFrame(rfc_grid.cv_results_)
rfc_df.shape

(1024, 22)

In [16]:
rfc_df.dropna(axis=0,how='any',inplace=True)

In [17]:
rfc_df.shape

(768, 22)

In [18]:
#rfc_df.to_csv('rfc_final.csv')

In [19]:
rfc_df.sample(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_max_features,param_max_leaf_nodes,param_min_samples_leaf,param_min_samples_split,...,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
737,0.724471,0.006492,0.058461,0.002755,entropy,6,sqrt,11,1,7,...,0.639752,0.65625,0.655603,0.012687,61,0.719626,0.719626,0.692547,0.710600,0.012765
242,1.149870,0.054439,0.099363,0.003273,gini,6,sqrt,16,1,12,...,0.664596,0.64375,0.657648,0.009827,57,0.753894,0.747664,0.754658,0.752072,0.003133
87,1.207306,0.086301,0.108365,0.015711,gini,1,sqrt,6,6,17,...,0.590062,0.58125,0.591266,0.008711,625,0.591900,0.598131,0.599379,0.596470,0.003271
624,0.659606,0.053757,0.062018,0.000035,entropy,1,sqrt,16,1,2,...,0.552795,0.59375,0.572658,0.016742,673,0.573209,0.588785,0.590062,0.584019,0.007662
319,0.722580,0.016298,0.067000,0.002160,gini,11,auto,16,16,17,...,0.627329,0.63750,0.641071,0.012928,415,0.660436,0.666667,0.661491,0.662864,0.002723


In [20]:
rfc_df.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_criterion', 'param_max_depth', 'param_max_features',
       'param_max_leaf_nodes', 'param_min_samples_leaf',
       'param_min_samples_split', 'params', 'split0_test_score',
       'split1_test_score', 'split2_test_score', 'mean_test_score',
       'std_test_score', 'rank_test_score', 'split0_train_score',
       'split1_train_score', 'split2_train_score', 'mean_train_score',
       'std_train_score'],
      dtype='object')

In [21]:
rfc_final_df = rfc_df.loc[:,['param_criterion','param_max_depth','param_max_features',
                            'param_max_leaf_nodes','param_min_samples_leaf','param_min_samples_split',
                            'mean_train_score','mean_test_score']]

In [22]:
rfc_final_df.to_csv('rfc_final.csv')

In [23]:
rfc_final_df.sample(5)

,param_criterion,param_max_depth,param_max_features,param_max_leaf_nodes,param_min_samples_leaf,param_min_samples_split,mean_train_score,mean_test_score
954,entropy,16,auto,16,11,12,0.682582,0.649379
915,entropy,16,auto,6,1,17,0.664948,0.643155
599,entropy,1,sqrt,6,6,17,0.584019,0.572658
890,entropy,11,sqrt,16,11,12,0.682582,0.649379
976,entropy,16,sqrt,6,1,2,0.665986,0.647296
